
# Mental Health Chatbot Training Notebook

This notebook provides a comprehensive pipeline to train a conversational mental health assistant. 
The system integrates emotion classification using `SamLowe/roberta-base-go_emotions` and text generation using `T5`.
It processes multiple cleaned datasets, performs training, evaluation, and finally builds a chatbot interface using Gradio or Streamlit.

## Objectives

- Load and preprocess multiple mental health-related datasets into a consistent question/answer format
- Simulate and encode multi-label emotion annotations using `MultiLabelBinarizer`
- Train a RoBERTa-based emotion classification model with live metric logging (accuracy, F1, precision, recall)
- Fine-tune two separate T5 models:
  - One for emotionally guided chatbot response generation
  - One for direct factual Q&A answering
- Implement emotion-aware routing logic that selects the appropriate model at inference time
- Create a unified RoBERTa + T5 pipeline for real-time response generation
- Build a Gradio chatbot interface using the full system
- Evaluate all models with live logging and inference testing
- Save all models and tokenizers in `./saved_models/`
- Save a `.pt` metadata file pointing to model paths for easy deployment


In [1]:
# Data Handling
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# PyTorch & Transformers
import torch
import torch.nn.functional as F
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

# Hugging Face Datasets & Evaluation
from datasets import load_dataset, Dataset, concatenate_datasets
from evaluate import load as load_metric

# External Evaluation Libraries
import evaluate  # for rouge, bertscore, and other NLP metrics

# Gradio for UI
import gradio as gr

# Progress Bar
from tqdm.auto import tqdm

# Optional (remove unless explicitly required)
# from accelerate import init_empty_weights




In [2]:
# Device setup (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Check device information
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Current Device Index:", torch.cuda.current_device())
    print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("Using CPU as GPU is not available.")

# Optional: set default tensor type to GPU-based FloatTensor if GPU is available
# Only uncomment if required
# if torch.cuda.is_available():
#     torch.set_default_dtype(torch.float32)
#     torch.set_default_tensor_type(torch.cuda.FloatTensor)


GPU Available: True
Current Device Index: 0
Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


## Load and Preprocess Datasets

In [3]:
import pandas as pd
from datasets import Dataset, concatenate_datasets

# Load and preprocess 'ds4'
ds4_path = './data/ds4_mental_health_chatbot_dataset_merged_modes.csv'

df = pd.read_csv(ds4_path)

# Drop the "mode" column if it exists
if 'mode' in df.columns:
    df.drop(columns=['mode'], inplace=True)
    print("Dropped 'mode' column.")
else:
    print("'mode' column not found.")

# Overwrite original CSV explicitly
df.to_csv(ds4_path, index=False)
print(f"Modified CSV saved and overwrote {ds4_path}.")

# File paths
dataset_paths = {
    "ds1": "./data/ds1_transformed_mental_health_chatbot_dataset.csv",
    "ds2": "./data/ds2_transformed_mental_health_chatbot.csv",
    "ds3": "./data/ds3_mental_health_faq_cleaned.csv",
    "ds4": ds4_path,
    "ds5": "./data/ds5_Mental_Health_FAQ.csv",
    "ds6": "./data/ds6_mental_health_counseling.csv"
}

# Enable/disable datasets
dataset_switches = {
    "ds1": False,
    "ds2": True,
    "ds3": False,
    "ds4": False,
    "ds5": False,
    "ds6": False
}

# Cleaning function
def load_and_clean_csv(path):
    df = pd.read_csv(path)
    df.columns = [col.lower().strip() for col in df.columns]

    # Rename columns to standard format
    if "prompt" in df.columns and "response" in df.columns:
        df.rename(columns={"prompt": "question", "response": "answer"}, inplace=True)
    if "questions" in df.columns:
        df.rename(columns={"questions": "question"}, inplace=True)
    if "answers" in df.columns:
        df.rename(columns={"answers": "answer"}, inplace=True)

    # Keep only necessary columns
    df = df[["question", "answer"]].dropna().reset_index(drop=True)
    
    return Dataset.from_pandas(df)

# Load selected datasets
datasets_list = []
for name, path in dataset_paths.items():
    if dataset_switches.get(name, False):
        dataset = load_and_clean_csv(path)
        print(f"Loaded dataset '{name}' with {len(dataset)} entries.")
        datasets_list.append(dataset)

# Validate that at least one dataset is loaded
if not datasets_list:
    raise ValueError("No datasets selected. Please enable at least one dataset in 'dataset_switches'.")

# Merge and split dataset
combined_dataset = concatenate_datasets(datasets_list).shuffle(seed=42)
split_dataset = combined_dataset.train_test_split(test_size=0.1)

train_ds = split_dataset["train"]
test_ds = split_dataset["test"]

# Display dataset sizes
print(f"Training dataset size: {len(train_ds)}")
print(f"Test dataset size: {len(test_ds)}")

train_ds, test_ds

'mode' column not found.
Modified CSV saved and overwrote ./data/ds4_mental_health_chatbot_dataset_merged_modes.csv.
Loaded dataset 'ds2' with 172 entries.
Training dataset size: 154
Test dataset size: 18


(Dataset({
     features: ['question', 'answer'],
     num_rows: 154
 }),
 Dataset({
     features: ['question', 'answer'],
     num_rows: 18
 }))

## Prepare 'labels' Column with MultiLabelBinarizer

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import random
import pandas as pd

# Set random seed for reproducibility
random.seed(42)

# Convert to DataFrame
df_train = train_ds.to_pandas()
df_test = test_ds.to_pandas()

# Simulated emotion annotations (replace with real data if available)
emotions_list = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
                 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
                 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization',
                 'relief', 'remorse', 'sadness', 'surprise']

df_train["emotions"] = [random.sample(emotions_list, k=random.randint(1, 3)) for _ in range(len(df_train))]
df_test["emotions"] = [random.sample(emotions_list, k=random.randint(1, 3)) for _ in range(len(df_test))]

# Encode with MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=emotions_list)

df_train["labels"] = list(mlb.fit_transform(df_train["emotions"]))
df_test["labels"] = list(mlb.transform(df_test["emotions"]))

# Quick sanity check
print("Sample emotion annotations (train):", df_train["emotions"].iloc[:3].tolist())
print("Sample binarized labels (train):", df_train["labels"].iloc[:3])

# Convert back to Hugging Face Dataset
train_ds = Dataset.from_pandas(df_train)
test_ds = Dataset.from_pandas(df_test)


Sample emotion annotations (train): [['annoyance', 'admiration', 'realization'], ['curiosity', 'approval'], ['annoyance', 'optimism', 'realization']]
Sample binarized labels (train): 0    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: labels, dtype: object


## Configure RoBERTa for Multi-Label Classification

In [5]:
from transformers import RobertaConfig, RobertaForSequenceClassification

# Define RoBERTa configuration explicitly for multi-label classification
config = RobertaConfig.from_pretrained(
    "SamLowe/roberta-base-go_emotions",
    problem_type="multi_label_classification",
    num_labels=len(emotions_list)
)

# Load the RoBERTa model with the defined configuration
model_emo = RobertaForSequenceClassification.from_pretrained(
    "SamLowe/roberta-base-go_emotions",
    config=config
).to(device)

print("RoBERTa multi-label classification model loaded and moved to device.")


RoBERTa multi-label classification model loaded and moved to device.


## Train RoBERTa with Correct Labels

In [6]:
import os
import torch
import numpy as np
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    RobertaConfig,
    DataCollatorWithPadding
)
from datasets import Features, Value, Sequence
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define model directory
model_path = "./saved_models/emotion_classifier"
os.makedirs(model_path, exist_ok=True)

# Set random seed for reproducibility
torch.manual_seed(42)

# Load tokenizer
tokenizer_emo = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

# Load or initialize model
if os.path.exists(os.path.join(model_path, 'config.json')):
    print("Loading previously trained model...")
    config = RobertaConfig.from_pretrained(model_path)
    model_emo = RobertaForSequenceClassification.from_pretrained(model_path, config=config).to(device)
else:
    print("No existing model found. Initializing new model.")
    config = RobertaConfig.from_pretrained(
        "SamLowe/roberta-base-go_emotions",
        problem_type="multi_label_classification",
        num_labels=len(emotions_list)
    )
    model_emo = RobertaForSequenceClassification.from_pretrained(
        "SamLowe/roberta-base-go_emotions",
        config=config
    ).to(device)

# Define dataset features
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "emotions": Sequence(Value("string")),
    "labels": Sequence(Value("float32"))
})

# Apply features format
train_ds = train_ds.cast(features)
test_ds = test_ds.cast(features)

# Tokenization function
def tokenize_emotion(example):
    enc = tokenizer_emo(
        example["question"],
        padding="max_length",
        truncation=True,
        max_length=128  # explicitly limit for efficiency
    )
    enc["labels"] = example["labels"]
    return enc

# Tokenize datasets
train_emo = train_ds.map(tokenize_emotion, batched=False)
test_emo = test_ds.map(tokenize_emotion, batched=False)

# Set format for PyTorch
train_emo.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_emo.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Data collator
collator = DataCollatorWithPadding(tokenizer=tokenizer_emo, return_tensors="pt")

# Metrics computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='micro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# Training arguments setup
training_args_emo = TrainingArguments(
    output_dir=model_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    report_to="none",
    save_safetensors=False,
    seed=42
)

# Initialize trainer
trainer_emo = Trainer(
    model=model_emo,
    args=training_args_emo,
    train_dataset=train_emo,
    eval_dataset=test_emo,
    tokenizer=tokenizer_emo,
    data_collator=collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer_emo.train()

# Save the trained model explicitly
trainer_emo.save_model(model_path)
tokenizer_emo.save_pretrained(model_path)

print(f"Model and tokenizer saved to {model_path}")


Loading previously trained model...


Casting the dataset:   0%|          | 0/154 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\mward\AppData\Local\Temp\ipykernel_33776\1231195333.py:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_emo = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.212100,0.231088,0.000000,0.000000,0.000000,0.000000
2,0.234000,0.239444,0.000000,0.000000,0.000000,0.000000
3,0.255100,0.227636,0.000000,0.000000,0.000000,0.000000


Model and tokenizer saved to ./saved_models/emotion_classifier


## Train T5 for Response Generation

In [7]:
import os
import shutil
import torch
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

# Define model save path
t5_response_path = "./saved_models/t5_response_generator"
os.makedirs(t5_response_path, exist_ok=True)

# Set random seed for reproducibility
torch.manual_seed(42)

# Cleanup invalid or empty model directory
if os.path.exists(t5_response_path) and not any(
    fname.endswith((".bin", ".safetensors", ".h5", ".index", ".msgpack"))
    for fname in os.listdir(t5_response_path)
):
    print("Empty or invalid model directory detected. Removing...")
    shutil.rmtree(t5_response_path)
    os.makedirs(t5_response_path, exist_ok=True)

# Load tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")

# Load or initialize T5 model explicitly on the correct device
if os.path.exists(os.path.join(t5_response_path, "pytorch_model.bin")):
    print("Loading existing trained T5 model...")
    model_t5_response = T5ForConditionalGeneration.from_pretrained(t5_response_path).to(device)
else:
    print("No existing T5 model found. Loading 't5-small'.")
    model_t5_response = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Tokenization function for T5 chat model
def tokenize_t5_chat(example):
    input_text = "chat: " + example["question"]
    target_text = example["answer"]

    model_inputs = tokenizer_t5(
        input_text,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer_t5(
        target_text,
        max_length=128,
        truncation=True,
        padding="max_length"
    ).input_ids

    model_inputs["labels"] = labels
    return model_inputs  

# Process datasets
train_chat = train_ds.map(tokenize_t5_chat, batched=False)
test_chat = test_ds.map(tokenize_t5_chat, batched=False)

# Set dataset format explicitly
train_chat.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_chat.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("Datasets processed and ready for training!")

# Data collator for seq2seq tasks (recommended for T5)
data_collator_seq2seq = DataCollatorForSeq2Seq(
    tokenizer=tokenizer_t5,
    model=model_t5_response,
    return_tensors="pt"
)

# Training arguments
training_args_chat = TrainingArguments(
    output_dir=t5_response_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    save_safetensors=False,
    report_to="none",
    seed=42
)

# Trainer setup and training
trainer_chat = Trainer(
    model=model_t5_response,
    args=training_args_chat,
    train_dataset=train_chat,
    eval_dataset=test_chat,
    tokenizer=tokenizer_t5,
    data_collator=data_collator_seq2seq
)

# Begin training
trainer_chat.train()

# Save model and tokenizer explicitly
trainer_chat.save_model(t5_response_path)
tokenizer_t5.save_pretrained(t5_response_path)

print("Training complete! T5 response-generation model and tokenizer saved.")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


No existing T5 model found. Loading 't5-small'.


Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Datasets processed and ready for training!


c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\mward\AppData\Local\Temp\ipykernel_33776\2789127582.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_chat = Trainer(
c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be remo

Epoch,Training Loss,Validation Loss
1,3.279200,1.709940
2,2.414200,1.602948
3,1.904200,1.573380


Training complete! T5 response-generation model and tokenizer saved.


## Train T5 for Q&A Assistant

In [8]:
import os
import torch
from transformers import (
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import numpy as np
import evaluate
from transformers import Seq2SeqTrainer


# Metric setup
metric = evaluate.load("rouge")

# Define model save path
t5_qa_path = "./saved_models/t5_qa"
os.makedirs(t5_qa_path, exist_ok=True)

# Load existing model or initialize new
if os.path.exists(os.path.join(t5_qa_path, "pytorch_model.bin")):
    print("Loading previously trained T5 Q&A model...")
    model_t5_qa = T5ForConditionalGeneration.from_pretrained(t5_qa_path).to(device)
else:
    print("No existing Q&A model found. Starting from 't5-small'.")
    model_t5_qa = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Tokenization function
def tokenize_t5_qa(examples):
    input_texts = ["question: " + q for q in examples["question"]]
    target_texts = examples["answer"]

    model_inputs = tokenizer_t5(
        input_texts, max_length=128, truncation=True, padding="max_length"
    )

    with tokenizer_t5.as_target_tokenizer():
        labels = tokenizer_t5(
            target_texts, max_length=128, truncation=True, padding="max_length"
        ).input_ids

    model_inputs["labels"] = labels
    return model_inputs


# Tokenize datasets
train_qa = train_ds.map(tokenize_t5_qa, batched=True)
test_qa = test_ds.map(tokenize_t5_qa, batched=True)

train_qa.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_qa.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Data collator
data_collator_qa = DataCollatorForSeq2Seq(
    tokenizer=tokenizer_t5,
    model=model_t5_qa,
    return_tensors="pt"
)

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Unpack predictions if necessary
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = [list(p) if isinstance(p, (np.ndarray, torch.Tensor)) else p for p in predictions]

    # Ensure labels is a list of lists
    if isinstance(labels[0], (int, np.integer)):
        labels = [labels]
    else:
        labels = [list(l) if isinstance(l, (np.ndarray, torch.Tensor)) else l for l in labels]

    # Replace -100 with pad_token_id
    labels = [[tokenizer_t5.pad_token_id if token == -100 else token for token in label] for label in labels]

    # Decode
    decoded_preds = tokenizer_t5.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer_t5.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: v for k, v in result.items()}




# Training arguments
training_args_qa = Seq2SeqTrainingArguments(
    output_dir=t5_qa_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    report_to="none",
    save_safetensors=False,
    seed=42,
    predict_with_generate=True  # 👈 key for Seq2Seq generation
)


# Trainer setup
trainer_qa = Seq2SeqTrainer(
    model=model_t5_qa,
    args=training_args_qa,
    train_dataset=train_qa,
    eval_dataset=test_qa,
    tokenizer=tokenizer_t5,
    data_collator=data_collator_qa,
    compute_metrics=compute_metrics
)

# Train and save
trainer_qa.train()
trainer_qa.save_model(t5_qa_path)
tokenizer_t5.save_pretrained(t5_qa_path)

print("Training complete! T5 Q&A model and tokenizer saved.")

No existing Q&A model found. Starting from 't5-small'.


Map:   0%|          | 0/154 [00:00<?, ? examples/s]

c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\mward\AppData\Local\Temp\ipykernel_33776\3986052619.py:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_qa = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.915700,1.882983,0.000000,0.000000,0.000000,0.000000
2,2.248400,1.759358,0.000000,0.000000,0.000000,0.000000
3,1.774300,1.722508,0.000000,0.000000,0.000000,0.000000


Training complete! T5 Q&A model and tokenizer saved.


## Unified Emotion-Aware Response System (RoBERTa + T5 Routing Logic)

In [9]:
import torch
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer,
    T5ForConditionalGeneration, T5Tokenizer
)

# Load models and tokenizers explicitly on the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

emotion_model_path = "./saved_models/emotion_classifier"
t5_chat_model_path = "./saved_models/t5_response_generator"
t5_qa_model_path = "./saved_models/t5_qa"

emotion_model = RobertaForSequenceClassification.from_pretrained(emotion_model_path).to(device)
emotion_tokenizer = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

t5_chat_model = T5ForConditionalGeneration.from_pretrained(t5_chat_model_path).to(device)
t5_qa_model = T5ForConditionalGeneration.from_pretrained(t5_qa_model_path).to(device)
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Emotion labels from GoEmotions dataset
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise'
]

# Emotion detection function
def detect_emotions(text, threshold=0.5):
    emotion_model.eval()
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze().cpu().tolist()
    detected = [(emotion_labels[i], p) for i, p in enumerate(probs) if p > threshold]
    return [label for label, _ in detected]

# Dynamic routing for response generation
def generate_combined_response(user_input):
    emotions = detect_emotions(user_input)

    emotional_keywords = {
        'joy', 'sadness', 'anger', 'fear', 'love', 'grief', 'remorse',
        'disappointment', 'gratitude', 'caring'
    }
    use_chat_model = any(e in emotional_keywords for e in emotions)

    if use_chat_model:
        prefix = "chat: "
        model = t5_chat_model
    else:
        prefix = "question: "
        model = t5_qa_model

    context = f"{' '.join(emotions)}: {user_input}" if emotions else user_input
    input_text = prefix + context

    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        output_ids = model.generate(inputs["input_ids"], max_length=128)
    response = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return {
        "Detected Emotions": emotions,
        "Model Used": "Chat Model" if use_chat_model else "QA Model",
        "Response": response
    }

# Test example
test_result = generate_combined_response("I feel really hopeless and angry all the time.")
print(test_result)


{'Detected Emotions': [], 'Model Used': 'QA Model', 'Response': ''}


## Save All Final Models and Tokenizers (RoBERTa + T5s)

In [10]:
import os
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer,
    T5ForConditionalGeneration, T5Tokenizer
)

# Define final model save directory
final_models_dir = "./saved_models/final_combined"
os.makedirs(final_models_dir, exist_ok=True)

# Load trained models explicitly from saved paths
chat_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_response_generator")
qa_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_qa")
emotion_model = RobertaForSequenceClassification.from_pretrained("./saved_models/emotion_classifier")

# Load respective tokenizers
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
emotion_tokenizer = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

# Save models to final directory
chat_model.save_pretrained(os.path.join(final_models_dir, "chat_model"))
qa_model.save_pretrained(os.path.join(final_models_dir, "qa_model"))
emotion_model.save_pretrained(os.path.join(final_models_dir, "emotion_model"))

# Save tokenizers explicitly
t5_tokenizer.save_pretrained(os.path.join(final_models_dir, "t5_tokenizer"))
emotion_tokenizer.save_pretrained(os.path.join(final_models_dir, "emotion_tokenizer"))

print(f"All models and tokenizers have been saved successfully in '{final_models_dir}'.")


All models and tokenizers have been saved successfully in './saved_models/final_combined'.


## Save Final Model Metadata (.pt) for Inference Pipeline

In [11]:
import torch
import os

# Define path explicitly and ensure directory exists
final_metadata_path = "./saved_models/final_model_metadata.pt"
os.makedirs(os.path.dirname(final_metadata_path), exist_ok=True)

# Lightweight metadata dictionary (pointer-style)
final_model_metadata = {
    "chat_model_path": "./saved_models/final/chat_model",
    "qa_model_path": "./saved_models/final/qa_model",
    "emotion_model_path": "./saved_models/final/emotion_model",
    "t5_tokenizer_path": "./saved_models/final/t5_tokenizer",
    "emotion_tokenizer_path": "./saved_models/final/emotion_tokenizer",
    "labels": emotion_labels  # list of emotion label names only
}

# Save the metadata dictionary
torch.save(final_model_metadata, final_metadata_path)

print(f"Final model metadata saved successfully at '{final_metadata_path}'.")



Final model metadata saved successfully at './saved_models/final_model_metadata.pt'.


## Gradio Chatbot Interface

In [12]:
import gradio as gr

# Chatbot interface function
def gradio_chat_interface(user_input, history):
    if history is None:
        history = []
    response_data = generate_combined_response(user_input)
    chatbot_response = response_data['Response']
    history.append((user_input, chatbot_response))
    return history, history

# Create Gradio interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs=[gr.Textbox(label="Your message"), gr.State()],
    outputs=[gr.Chatbot(label="Chat History"), gr.State()],
    title="Mental Health Chatbot - Happy Brain",
    description="Emotion-aware chatbot using RoBERTa + T5",
    allow_flagging="never"
)

interface.launch()



C:\Users\mward\AppData\Local\Temp\ipykernel_33776\1799456066.py:16: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  outputs=[gr.Chatbot(label="Chat History"), gr.State()],
c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Model Evaluation Metrics and Sample Inference

In [13]:
import evaluate
import numpy as np
import torch

# Load evaluation metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Prepare sample data
sample_size = min(100, len(test_ds))
sample = test_ds.select(range(sample_size))

# Function to generate predictions
def generate_predictions(model, tokenizer, dataset, device):
    inputs = [f"question: {x['question']}" for x in dataset]
    inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).to(device)
    output_ids = model.generate(inputs['input_ids'], max_length=128)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    refs = [x["answer"] for x in dataset]
    return preds, refs

# Generate predictions for evaluation
predictions, references = generate_predictions(model_t5_qa, tokenizer_t5, sample, device)

# Evaluate using ROUGE and BERTScore
rouge_results = rouge.compute(predictions=predictions, references=references)
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")

# Display evaluation results
print("ROUGE Scores:", rouge_results)
print("BERTScore:", {k: np.mean(v) for k, v in bertscore_results.items() if k != 'hashcode'})

# RoBERTa Emotion Classifier - Inference Test
sample_input = "I feel like I'm breaking down and can't handle anything."

# Properly tokenize for RoBERTa and move to device
inputs = emotion_tokenizer(sample_input, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

emotion_model.eval()
with torch.no_grad():
    logits = emotion_model(**inputs).logits
    probs = torch.sigmoid(logits).squeeze().cpu().tolist()

    predicted_emotions = [
        emotion_labels[i] for i, p in enumerate(probs) if p > 0.5
    ]

print("Detected Emotions:", predicted_emotions)

# T5 QA Model - Inference Test
qa_input = "question: What are some ways to manage daily anxiety?"
inputs = tokenizer_t5(qa_input, return_tensors="pt").to(device)

qa_model.eval()
with torch.no_grad():
    output_ids = qa_model.generate(inputs["input_ids"], max_length=128)

response = tokenizer_t5.decode(output_ids[0], skip_special_tokens=True)
print("QA Model Response:", response)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE Scores: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
BERTScore: {'precision': 0.0, 'recall': 0.7806296116775937, 'f1': 0.0}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)